In [1]:
!pip install findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType , StructField, StringType, IntegerType

In [2]:
#RDD (Esnek Dagıtılmış Veri Kümesi)
#(harita filtre indirgeyici birleştirme) mapping filan
#hdfs filan ilede çalışıyor
#tranformation(dönüşümler mapping filan) action(eylemler count() sum()) 
#spark tutortial tr de var hepsi

In [3]:
spark = SparkSession.builder.appName("LinearRegerssion with pyspark").getOrCreate()
#arkada deltalake çalışıyor o yüzden datawarehouse ile datalake birlikte kullanımı deltalake
df=spark.read.format('delta').options(header="True",inferschema="True").csv("C:\\Users\\brhng\\Desktop\\Advertising.csv",header=True)

In [4]:
df.printSchema()
df.show(5)

root
 |-- _c0: integer (nullable = true)
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)

+---+-----+-----+---------+-----+
|_c0|   TV|Radio|Newspaper|Sales|
+---+-----+-----+---------+-----+
|  1|230.1| 37.8|     69.2| 22.1|
|  2| 44.5| 39.3|     45.1| 10.4|
|  3| 17.2| 45.9|     69.3|  9.3|
|  4|151.5| 41.3|     58.5| 18.5|
|  5|180.8| 10.8|     58.4| 12.9|
+---+-----+-----+---------+-----+
only showing top 5 rows



In [5]:
df = df.drop("_c0")
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|               0.0|               0.3|               1.6|
|    max|            296.4|              49.6|             114.0|              27.0|
+-------+-----------------+------------------+------------------+------------------+



In [6]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

def transData(data):
    return data.rdd.map(lambda r : [Vectors.dense(r[:-1]),r[-1]]).toDF(["features","label"])
    #
transformed = transData(df)
transformed.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

#kategorik bir değişkenimiz varsa teyit amaçlı onu dönüştürüyor indexleyerek
featureIndexer = VectorIndexer(inputCol = "features",outputCol="indexedFatures",maxCategories=4).fit(transformed)
data= featureIndexer.transform(transformed)
data.show(5)

+-----------------+-----+-----------------+
|         features|label|   indexedFatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



In [8]:
(trainingData,testData) = data.randomSplit([0.6,0.4])
trainingData.show(5)
testData.show(5)

+----------------+-----+----------------+
|        features|label|  indexedFatures|
+----------------+-----+----------------+
|  [0.7,39.6,8.7]|  1.6|  [0.7,39.6,8.7]|
|  [4.1,11.6,5.7]|  3.2|  [4.1,11.6,5.7]|
|  [8.4,27.2,2.1]|  5.7|  [8.4,27.2,2.1]|
|[11.7,36.9,45.2]|  7.3|[11.7,36.9,45.2]|
|[13.2,15.9,49.6]|  5.6|[13.2,15.9,49.6]|
+----------------+-----+----------------+
only showing top 5 rows

+---------------+-----+---------------+
|       features|label| indexedFatures|
+---------------+-----+---------------+
| [5.4,29.9,9.4]|  5.3| [5.4,29.9,9.4]|
|[7.3,28.1,41.4]|  5.5|[7.3,28.1,41.4]|
|[7.8,38.9,50.6]|  6.6|[7.8,38.9,50.6]|
|  [8.6,2.1,1.0]|  4.8|  [8.6,2.1,1.0]|
|[8.7,48.9,75.0]|  7.2|[8.7,48.9,75.0]|
+---------------+-----+---------------+
only showing top 5 rows



In [9]:
lr = LinearRegression()
pipeline = Pipeline(stages=[featureIndexer,lr])
model = pipeline.fit(trainingData)

In [10]:
predictions = model.transform(testData)
predictions.select("features","label","prediction").show(5)

+---------------+-----+------------------+
|       features|label|        prediction|
+---------------+-----+------------------+
| [5.4,29.9,9.4]|  5.3| 8.637037975543508|
|[7.3,28.1,41.4]|  5.5| 8.797242769981187|
|[7.8,38.9,50.6]|  6.6|10.958416521656462|
|  [8.6,2.1,1.0]|  4.8|  3.47261409550541|
|[8.7,48.9,75.0]|  7.2| 13.18333585852772|
+---------------+-----+------------------+
only showing top 5 rows



In [11]:
evaluator = RegressionEvaluator(labelCol="label",predictionCol="prediction",metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE:",rmse)

RMSE: 1.664356884258002


In [12]:
#r0.5 dahada dengeli küçük verilerde
#r1 dengeli ike
#r2 truepostivler cogunlukta ise
#r2 score
y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics

r2_score = sklearn.metrics.r2_score(y_true,y_pred)
print("r2-score:",r2_score)

r2-score: 0.877379371624068


In [13]:
#Decision Tree Classification

In [43]:
spark = SparkSession.builder.appName("LinearRegerssion with pyspark").getOrCreate()
#arkada deltalake çalışıyor o yüzden datawarehouse ile datalake birlikte kullanımı deltalake
df=spark.read.format('delta').options(header="True",inferschema="True").csv("C:\\Users\\brhng\\Desktop\\WineData.csv",header=True)

In [44]:
df.show(5)

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      5|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [45]:
df.limit(5).toPandas()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [46]:
def condition(r):
    if(0 <= r <=4):
        label ="low"
    elif(4<r<=6):
        label="medium"
    else:
        label="high"
    return label

def string_to_float(x):
    return float(x)


In [47]:
from pyspark.sql.functions import udf #kullanıcı tanımlı dosya tipi (udf)
from pyspark.sql.types import StringType, DoubleType
string_to_float_udf = udf(string_to_float,DoubleType())
quality_udf = udf(lambda x:condition(x),StringType()) #fonksiyonlar ve hangi type karlışılık gelmesini bekldigimiz veriyoruz

In [48]:
df = df.withColumn("quality",quality_udf("quality"))
df.show(5)
df.printSchema()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|quality|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4| medium|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8| medium|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8| medium|
|         11.2|            0.28|       0.56|           1.9|    0.075|               17.0|           

In [49]:
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer,StringIndexer,IndexToString
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
def transData(data):
    return data.rdd.map(lambda r : [Vectors.dense(r[:-1]),r[-1]]).toDF(["features","label"])

transformed = transData(df)
transformed.show(5)

+--------------------+------+
|            features| label|
+--------------------+------+
|[7.4,0.7,0.0,1.9,...|medium|
|[7.8,0.88,0.0,2.6...|medium|
|[7.8,0.76,0.04,2....|medium|
|[11.2,0.28,0.56,1...|medium|
|[7.4,0.7,0.0,1.9,...|medium|
+--------------------+------+
only showing top 5 rows



In [51]:
from pyspark.ml.feature import VectorIndexer,StringIndexer,IndexToString
labelIndexer =  StringIndexer(inputCol ="label",outputCol="indexedLabel").fit(transformed)
labelIndexer.transform(transformed).show(5)

featureIndexer = VectorIndexer(inputCol="features",outputCol="indexedFeatures",maxCategories=4).fit(transformed)
featureIndexer.transform(transformed).show(5)

+--------------------+------+------------+
|            features| label|indexedLabel|
+--------------------+------+------------+
|[7.4,0.7,0.0,1.9,...|medium|         0.0|
|[7.8,0.88,0.0,2.6...|medium|         0.0|
|[7.8,0.76,0.04,2....|medium|         0.0|
|[11.2,0.28,0.56,1...|medium|         0.0|
|[7.4,0.7,0.0,1.9,...|medium|         0.0|
+--------------------+------+------------+
only showing top 5 rows

+--------------------+------+--------------------+
|            features| label|     indexedFeatures|
+--------------------+------+--------------------+
|[7.4,0.7,0.0,1.9,...|medium|[7.4,0.7,0.0,1.9,...|
|[7.8,0.88,0.0,2.6...|medium|[7.8,0.88,0.0,2.6...|
|[7.8,0.76,0.04,2....|medium|[7.8,0.76,0.04,2....|
|[11.2,0.28,0.56,1...|medium|[11.2,0.28,0.56,1...|
|[7.4,0.7,0.0,1.9,...|medium|[7.4,0.7,0.0,1.9,...|
+--------------------+------+--------------------+
only showing top 5 rows



In [52]:
(trainingData,testData) = transformed.randomSplit([0.6,0.4])
trainingData.show(5)
testData.show(5)

+--------------------+------+
|            features| label|
+--------------------+------+
|[4.6,0.52,0.15,2....|   low|
|[4.9,0.42,0.0,2.1...|  high|
|[5.0,0.4,0.5,4.3,...|medium|
|[5.1,0.42,0.0,1.8...|  high|
|[5.1,0.47,0.02,1....|medium|
+--------------------+------+
only showing top 5 rows

+--------------------+------+
|            features| label|
+--------------------+------+
|[4.7,0.6,0.17,2.3...|medium|
|[5.0,0.38,0.01,1....|medium|
|[5.0,0.42,0.24,2....|  high|
|[5.0,0.74,0.0,1.2...|medium|
|[5.0,1.02,0.04,1....|   low|
+--------------------+------+
only showing top 5 rows



In [53]:
from pyspark.ml.classification import DecisionTreeClassifier

dTree = DecisionTreeClassifier(labelCol="indexedLabel",featuresCol="indexedFeatures")

In [59]:
from pyspark.ml.feature import VectorIndexer,StringIndexer,IndexToString
#Pipeline 
labelConverter = IndexToString(inputCol = "prediction",outputCol="predictidLabel",labels=labelIndexer.labels)
pipeline = Pipeline(stages=[labelIndexer,featureIndexer,dTree,labelConverter])
model = pipeline.fit(trainingData)
predictions = model.transform(testData)
predictions.select("features","label","prediction").show(5)

+--------------------+------+----------+
|            features| label|prediction|
+--------------------+------+----------+
|[4.7,0.6,0.17,2.3...|medium|       0.0|
|[5.0,0.38,0.01,1....|medium|       1.0|
|[5.0,0.42,0.24,2....|  high|       1.0|
|[5.0,0.74,0.0,1.2...|medium|       0.0|
|[5.0,1.02,0.04,1....|   low|       0.0|
+--------------------+------+----------+
only showing top 5 rows



In [63]:
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel",predictionCol="prediction",metricName="accurarcy")
accurarcy = evaluator.evaluate(predictions)
print("Test Error = %g"%(1-0-accuracy))
rfModel = model.stages[-2]
print(rfModel)

IllegalArgumentException: MulticlassClassificationEvaluator_ff94deea73f3 parameter metricName given invalid value accurarcy.

In [65]:
from sklearn.metrics import confusion_matrix

y_true = predictions.select("label")
y_true = y_true.toPandas()

y_pred = predictions.select("predictedLabel")
y_pred = y_pred.toPandas()

cnf_matrix = confusion_matrix(y_true,y_pred)
cnf_matrix

AnalysisException: cannot resolve '`predictedLabel`' given input columns: [features, indexedFeatures, indexedLabel, label, predictidLabel, prediction, probability, rawPrediction];;
'Project ['predictedLabel]
+- Project [features#1543, label#1544, indexedLabel#1823, indexedFeatures#1828, rawPrediction#1834, probability#1841, prediction#1849, UDF(cast(prediction#1849 as double)) AS predictidLabel#1864]
   +- Project [features#1543, label#1544, indexedLabel#1823, indexedFeatures#1828, rawPrediction#1834, probability#1841, UDF(rawPrediction#1834) AS prediction#1849]
      +- Project [features#1543, label#1544, indexedLabel#1823, indexedFeatures#1828, rawPrediction#1834, UDF(rawPrediction#1834) AS probability#1841]
         +- Project [features#1543, label#1544, indexedLabel#1823, indexedFeatures#1828, UDF(indexedFeatures#1828) AS rawPrediction#1834]
            +- Project [features#1543, label#1544, indexedLabel#1823, UDF(features#1543) AS indexedFeatures#1828]
               +- Project [f